
![JupyterGames](./JupyterGames.png)


# Introduction

Making simple video games has always been a great way to learn programming for beginner.
While jupyter notebooks are the defacto standard for interactive programming in data science and machine learning, they are seldomly used forgame development.
In this blogpost we will explore the difficulties in developing simple games in jupyter notebooks and how to overcome them.

## The canvas

A drawable surface is the most basic requirement for any game development. In the web world, this is usually provided by the HTML5 `<canvas>` element.
In Jupyter, there are several ways to create and manipulate a canvas. The most popular one is the `ipycanvas` package, which provides a high-level API for creating and manipulating canvases in Jupyter notebooks.
However, `ipycanvas` is tied to the jupyter protocol, which makes it difficult to create a smooth gaming experience.

The jupyter protocol is a fire and forget protocol. Once a cell is executed, the kernel sends the output to the frontend and forgets about it.
This works well for data science and machine learning, where the output is usually a static image or text.

While there exisits interacive widgets, the interactivitiy is usually not bidirectional from within a single cell, while for games we precisely need that: The kernel sends updates to the frontend, ie what to render, and the frontend sends back user input (keyboard, mouse, touch) to the kernel, and this needs to happen continuously at a fixed frame rate from a running cell in a kernel.
While we can archieve this single-cell-bidirectional-interactivity with `ipywidgets` and `ipycanvas`, via asynchronous programming, there are still some limitations:

While `ipycanvas` provides a way to update the canvas, it is impossible with ipycanvas to know when the frontend has actually rendered the updated canvas. This will lead to situations where the kernel is sending updates faster than the frontend can render them, leading to a choppy and unresponsive gaming experience.


## JupiterLite

[JupyterLite](https://jupyterlite.readthedocs.io/en/latest/) is a Jupyter distribution that runs entirely in the browser. It uses kernels compiled to WebAssembly like xeus-python, pyodide, xeus-r and even xeus-cpp, to run code directly in the browser without any server backend.
Jupyterlite supports widgets like `ipywidgets` and `ipycanvas` but is also tied to the jupyter protocol.

##

## The OffscreenCanvas 

The HTML5 OffscreenCanvas API provides a way to create a canvas that can be rendered offscreen, i.e., without being attached to the DOM. This allows us to create a canvas in the frontend / main-ui thread, and then transfer it to a web worker for rendering. 
Using the OffscreenCanvas API, we can create a custom canvas widget that creates an OffscreenCanvas in the frontend, transfers it to the kernel running in a web worker, and then uses it for rendering.
Since kernels like xeus-python provide a Javascript foreign function interface, we can directly call Javascript functions from Python , and use them to directely call the OffscreenCanvas API. Hence we can create a smooth gaming experience and render to the canvas at a fixed frame rate. That way, it is impossible to send updates faster than the frontend can render them, since the calls to the OffscreenCanvas API are blocking until the rendering is done.
That way we can bypass the limitations of the jupyter protocol and create a smooth gaming experience in JupyterLite.

We integrated the OffscreenCanvas API directely into the `ipycanvas` package and created a custom canvas widget called `OffscreenCanvas` which provides the same API as the regular `Canvas` widget from `ipycanvas`, but uses the OffscreenCanvas API for rendering.



## Overcome the protocol limitations for regular Jupyter

Since the OffscreenCanvas API is not available in regular Jupyter notebooks, we need to find a way to overcome the limitations of the jupyter protocol.
While its impossible with ipycanvas existing API to know when the canvas has been rendered in the frontned for the reason mentioned above, we can work around this limitation by creating a custom frontend extension that uses the following trick.

Asuming we want to know within a the exection of a single cell when some value of a widget has changed in the frontend,
we can utilize the following snippet that creates an asyncio.Future that will be set once the value of the widget changes in the frontend.

```python
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future
```

As an example, the following code creates an IntSlider widget and an async function that waits for the value of the slider to change.

```python
from ipywidgets import IntSlider
slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider
```

In [ ]:
import asyncio
from ipywidgets import IntSlider

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

Using the above trick, we can create a custom canvas widget that will notify the kernel once the canvas has been rendered in the frontend. This way, we can create a game loop that runs at a fixed frame rate and only updates the canvas once the previous frame has been rendered.

# Box2d

[Box2D](https://box2d.org/) is a popular 2D physics engine that is widely used in game development. It provides a simple and efficient way to simulate physics in 2D games.
Recently, there has been a major update to the Box2D codebase, namely Box2D 3.0 which introduces several new features and dramatic performance improvements over the previous versions.

## Python wrapper for Box2D 3.0
We have created a Python wrapper for Box2D 3.0 called [pyb2d3](https://github.com/DerThorsten/pyb2d3/) that allows you to use Box2D 3.0 in Python. The wrapper is built using [nanobind](https://nanobind.github.io/) and provides a simple and efficient way to use Box2D 3 in Python.
To be able to use pyb2d3 in jupyterlite  we created a wasm build of pyb2d3 using [emscripten](https://emscripten.org/) and the  [emscripten-forge](https://emscripten-forge.org/) conda [distribution](https://prefix.dev/channels/emscripten-forge-dev).

## Box2D sandbox

Box2d and its python wrapper pyb2d3 do not provide any rendering capabilities out of the box. However both Box2D and pyb2d3 provide a "sandbox" playground that allows you to quickly create simple Box2D which are automatically rendered.

For pyb2d3, we have created a custom package for this sandbox called `pyb2d3-sandbox` with various frontends, including a jupyter notebook frontend that uses the custom canvas widget described above to provide a smooth gaming experience in jupyter notebooks.

These frontends are:

 * pyb2d3-sandbox-opengl: A desktop OpenGL frontend for pyb2d3-sandbox using PyOpenGL and dearimgui
 * pyb2d3-sandbox-pygame: A desktop pygame frontend for pyb2d3-sandbox using pygame for rendering
 * pyb2d3-sandbox-ipycanvas: A Jupyter notebook frontend for pyb2d3-sandbox using the custom OffscreenCanvas widget when in JupyterLite
 * pyb2d3-sandbox-jupyter: A Jupyter notebook frontend for pyb2d3-sandbox using `wait_for_change` trick when in regular Jupyter notebooks


# Examples

All the examples can also be found, edited and run in [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/)

## Netwons Cradle

Lets start with the classic Newtons Cradle example. The example can be restarted by pressing the stop-like button in the lower left corner of the canvas.
Mouse or touch interaction can be used to drag the balls arround.

```python
world = b2d.World(gravity=(0, -10))
n_balls = 10
radius = 1
actual_radius = radius * 0.85 # make balls a  bit smaller st. 
                              # there is a little space between them
rope_length = 10
diameter = radius * 2

for i in range(n_balls):
    x = diameter * i
    y_ball = 0
    y_rope = rope_length

    # create dynamic body for the ball
    ball_body = world.create_dynamic_body(
        position=(x, y_ball), linear_damping=0.1, angular_damping=0.0
    )
    ball_body.awake = True
    # create circle shape for the ball
    material = b2d.surface_material(
        restitution=1.0,
        friction=0.0,
        custom_color=b2d.hex_color(100, 0, 200),
    )
    ball_body.create_shape(
        b2d.shape_def(density=1, material=material),
        b2d.circle(radius=actual_radius),
    )

    # create a rope anchor for the balls
    anchor_pos = (x, y_rope)
    anchor_body_id = world.create_static_body(position=anchor_pos)

    self.world.create_distance_joint(
        body_a=ball_body,
        body_b=anchor_body_id,
        length=self.rope_length,
        enable_spring=False,
    )

impulse = (-10, 0)
ball_body.apply_linear_impulse_to_center(impulse, wake=True)
```

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase


class NewtonsCradle(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        # physical world
        self.n_balls = 10
        self.radius = 1
        self.actual_radius = self.radius * 0.85
        self.rope_length = 10

        diameter = self.radius * 2
        for i in range(self.n_balls):
            x = diameter * i
            y_ball = 0
            y_rope = self.rope_length

            # create dynamic body for the ball
            ball_body = self.world.create_dynamic_body(
                position=(x, y_ball), linear_damping=0.1, angular_damping=0.0
            )
            ball_body.awake = True
            # create circle shape for the ball
            material = b2d.surface_material(
                restitution=1.0,
                friction=0.0,
                custom_color=b2d.hex_color(100, 0, 200),
            )
            ball_body.create_shape(
                b2d.shape_def(density=1, material=material),
                b2d.circle(radius=self.actual_radius),
            )

            # create a rope anchor for the balls
            anchor_pos = (x, y_rope)
            anchor_body_id = self.world.create_static_body(position=anchor_pos)

            self.world.create_distance_joint(
                body_a=ball_body,
                body_b=anchor_body_id,
                length=self.rope_length,
                enable_spring=False,
            )

        impulse = (-10, 0)
        ball_body.apply_linear_impulse_to_center(impulse, wake=True)

    def aabb(self):
        return b2d.aabb(
            lower_bound=(-(self.rope_length + 2 * self.radius), 0),
            upper_bound=(
                self.n_balls * self.radius * 2 + self.rope_length,
                self.rope_length + 2 * self.radius,
            ),
        )


if __name__  == "__main__":
    NewtonsCradle.run(frontend_settings=dict(simple_ui=True, autostart=True))

## Billiard

The source code of the billiard example can be found in `billiard.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/) deployment.
To start the example, press the play button. Note that this will pause the simulation of all other running examples.
To shoot a ball, click into a ball and drag the mouse / finger backwards. On release, the ball will shot.

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase

import numpy as np
import math

# from dataclass  import dataclass
from dataclasses import dataclass
import random

import enum


@dataclass
class Ball:
    color: tuple[int, int, int]
    is_half: bool = False
    is_white: bool = False
    is_black: bool = False
    body: b2d.Body = None


# enum for state of game
class GameState(enum.Enum):
    WAITING_FOR_BALL_SELECTION = enum.auto()
    WAITING_FOR_SHOT = enum.auto()
    WAITING_FOR_BALLS_TO_REST = enum.auto()


class Billiard(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings.set_gravity((0, 0)))

        # billiard table
        w = 8
        h = w / 2
        self.width_lower_bound = w
        self.height_lower_bound = h

        # top_left_corner
        top_left_corner = (-w / 2, h / 2)

        # pocket radius
        r = 0.5
        self.pocket_radius = r

        md = r * math.sqrt(2) * 1.3
        cd = r * 1.3

        self.pocket_centers = []

        # start with left pocket arc
        start = (top_left_corner[0], top_left_corner[1])
        builder = b2d.PathBuilder(start)
        pocket_center = builder.arc_to(
            delta=(-cd, -cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move down by height
        builder.line_to(down=h)

        # add bottom left corner pocket
        pocket_center = builder.arc_to(
            delta=(cd, -cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move right by width /2
        builder.line_to(right=w / 2)
        # add bottom middle pocket
        pocket_center = builder.arc_to(
            delta=(md, 0), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move right by width /2
        builder.line_to(right=w / 2)
        # add bottom right corner pocket
        pocket_center = builder.arc_to(
            delta=(cd, cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move up by height
        builder.line_to(up=h)
        # add top right corner pocket
        pocket_center = builder.arc_to(
            delta=(-cd, cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move left by width/2
        builder.line_to(left=w / 2)
        # add top middle pocket
        pocket_center = builder.arc_to(
            delta=(-md, 0), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)
        # move left by width/2
        builder.line_to(left=w / 2)

        assert len(self.pocket_centers) == 6, "There should be 6 pocket centers"

        table_center = (np.array(self.pocket_centers[2]) + np.array(self.pocket_centers[5])) / 2
        table_center = (float(table_center[0]), float(table_center[1]))
        self.table_center = table_center

        # add 15 balls
        self.balls = []
        self.ball_radius = 0.2

        def create_ball(position, color):
            material = b2d.surface_material(
                custom_color=b2d.hex_color(color[0], color[1], color[2]),
                restitution=1.0,
            )
            ball = self.world.create_dynamic_body(
                position=position, linear_damping=0.8, fixed_rotation=True
            )
            ball.create_shape(b2d.shape_def(material=material), b2d.circle(radius=self.ball_radius))
            return ball

        billiard_base_colors = [
            (255, 255, 0),  # 1 - Yellow
            (0, 0, 255),  # 2 - Blue
            (255, 0, 0),  # 3 - Red
            (128, 0, 128),  # 4 - Purple
            (255, 165, 0),  # 5 - Orange
            (0, 128, 0),  # 6 - Green
            (128, 0, 0),  # 7 - Maroon (Burgundy)
        ]
        # create a list of all balls (except white and black)
        for i, color in enumerate(billiard_base_colors):
            self.balls.append(Ball(color=color, is_half=False))
            self.balls.append(Ball(color=color, is_half=True))

        # shuffle the balls
        random.shuffle(self.balls[1:])  # keep the first ball (yellow) in place

        self.balls.insert(4, Ball(color=(0, 0, 1), is_black=True))

        # add a white ball as last ball
        self.balls.append(Ball(color=(255, 255, 255), is_white=True))

        ball_index = 0
        for x in range(5):
            n_balls = x + 1
            # pos_x = x * self.ball_radius *
            pos_x = x * self.ball_radius * math.sqrt(3)
            offset_y = -x * self.ball_radius

            for y in range(n_balls):
                ball_item = self.balls[ball_index]
                pos_y = offset_y + y * self.ball_radius * 2
                ball_body = create_ball(
                    position=(table_center[0] + pos_x, table_center[1] + pos_y),
                    color=ball_item.color,
                )
                self.balls[ball_index].body = ball_body
                ball_index += 1

        # white ball
        white_ball = create_ball(
            position=(table_center[0] - w / 4, table_center[1]),
            color=self.balls[-1].color,
        )
        self.balls[-1].body = white_ball

        self.outline = np.array(builder.points)

        material = b2d.surface_material(
            restitution=1.0,
            custom_color=b2d.hex_color(10, 150, 10),
        )
        self.anchor_body.create_chain(
            builder.chain_def(is_loop=True, reverse=True, material=material)
        )

        # state of the game
        self.game_state = GameState.WAITING_FOR_BALL_SELECTION
        self.marked_point_on_white_ball = None
        self.aim_point = None

        # in case of a headless frontend we do one shot
        if self.frontend.settings.headless:
            # we create a mouse joint for the white ball
            self.balls[-1].body.apply_linear_impulse_to_center((3.5, 0.1), wake=True)
            self.game_state = GameState.WAITING_FOR_BALLS_TO_REST

    def on_mouse_down(self, event):
        if self.game_state == GameState.WAITING_FOR_BALL_SELECTION:
            # check if mouse is over **the white ball**
            ball_shape = self.balls[-1].body.shapes[0]
            world_pos = event.world_position
            if ball_shape.test_point(point=world_pos):
                self.marked_point_on_white_ball = world_pos
                self.aim_point = world_pos
                self.game_state = GameState.WAITING_FOR_SHOT

    def on_mouse_move(self, event):
        if self.game_state == GameState.WAITING_FOR_SHOT:
            self.aim_point = event.world_position

    def on_mouse_up(self, event):
        if self.game_state == GameState.WAITING_FOR_SHOT:
            # shoot the white ball
            # get the impulse vector from the marked point on the white ball to the force selection point
            ball_pos = self.balls[-1].body.position
            impulse_vec = -(np.array(self.aim_point) - np.array(ball_pos))
            impulse_vec = (
                float(impulse_vec[0]),
                float(impulse_vec[1]),
            )  # convert to tuple
            # apply the force as impulse
            self.balls[-1].body.apply_linear_impulse(impulse_vec, self.marked_point_on_white_ball)
            self.game_state = GameState.WAITING_FOR_BALLS_TO_REST
            self.marked_point_on_white_ball = None
            self.aim_point = None

    def pre_update(self, dt):
        if self.game_state == GameState.WAITING_FOR_BALLS_TO_REST:
            all_rest = True
            for ball in self.balls:
                body = ball.body
                mag = body.linear_velocity_magnitude()
                if mag > 0.0001:  # threshold for resting
                    all_rest = False
                    break
            if all_rest:
                self.game_state = GameState.WAITING_FOR_BALL_SELECTION
                self.marked_point_on_white_ball = None
                self.aim_point = None

        # check if any ball is in a pocket
        to_be_removed = []
        dr = self.pocket_radius - self.ball_radius
        for ball in self.balls:
            body = ball.body
            for pocket_center in self.pocket_centers:
                if np.linalg.norm(np.array(body.position) - np.array(pocket_center)) < dr:
                    to_be_removed.append(ball)
                    break

        for ball in to_be_removed:
            if ball.is_white or ball.is_black:
                self.frontend.set_sample(type(self), self.settings)
            if self.mouse_joint_body is not None:
                if ball.body == self.mouse_joint_body:
                    self.destroy_mouse_joint()
            self.balls.remove(ball)
            ball.body.destroy()

    def pre_debug_draw(self):
        for pocket_center in self.pocket_centers:
            t = b2d.transform(pocket_center)
            self.debug_draw.draw_solid_circle(
                transform=t,
                radius=0.5,
                color=(0, 0, 0),
            )

        # self.debug_draw.draw_polygon(points=self.outline, color=(10, 150, 10))

    def post_debug_draw(self):
        for ball in self.balls:
            self.debug_draw.draw_solid_circle(
                transform=ball.body.transform,
                radius=self.ball_radius,
                color=ball.color,
            )
            if ball.is_half:
                self.debug_draw.draw_solid_circle(
                    transform=ball.body.transform,
                    radius=self.ball_radius / 2,
                    color=(255, 255, 255),
                )
        if self.marked_point_on_white_ball is not None:
            self.debug_draw.draw_solid_circle(
                transform=b2d.transform(self.marked_point_on_white_ball),
                radius=0.1,
                color=(255, 0, 0),
            )

            if self.aim_point is not None:
                self.debug_draw.draw_segment(
                    self.marked_point_on_white_ball, self.aim_point, color=(255, 0, 0)
                )

    def aabb(self):
        hx = self.width_lower_bound / 2
        hy = self.height_lower_bound / 2
        margin = hx * 1.5
        center = self.table_center
        return b2d.aabb(
            lower_bound=(center[0] - (hx + margin), center[1] - (hy + margin)),
            upper_bound=(center[0] + (hx + margin), center[1] + (hy + margin)),
        )


if __name__  == "__main__":
    Billiard.run(frontend_settings=dict(simple_ui=True, autostart=False))

## Ragdoll

While this is not a game, its still fun to man-handle the ragdoll using mouse or touch interaction.
A double click will trigger an explosion that will fling the ragdoll parts arround. A tripple click will trigger an implosion that will suck the ragdoll parts towards the click position.
The source code of this example can be found in `ragdoll.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/)

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase
import random

from examples_common import Ragdoll as RagdollComposit


class Ragdoll(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        self.outer_box_radius = 30
        self.box_body = self.world.create_static_body(position=(0, 0))
        self.box_body.create_chain(
            b2d.chain_box(center=(0, 0), hx=self.outer_box_radius, hy=self.outer_box_radius)
        )

        def rand_pos():
            margin = 2
            r = self.outer_box_radius - margin
            return (random.uniform(-r, r), random.uniform(-r, 0))

        num_bodies = 15
        for _ in range(num_bodies):
            # ragdoll at the center
            self.ragdoll = RagdollComposit(
                scale=7.0,
                world=self.world,
                position=rand_pos(),
                colorize=True,
                hertz=0.1,
            )

        # only relevant for a headless ui
        self._exploded = False

    def pre_update(self, dt):
        if self.frontend.settings.headless and self.world_time > 2 and not self._exploded:
            self._exploded = True
            self.world.explode(
                position=(0, -self.outer_box_radius), radius=20, impulse_per_length=30
            )

    def on_double_click(self, event):
        self.world.explode(position=event.world_position, radius=20, impulse_per_length=30)

    def on_triple_click(self, event):
        self.world.explode(position=event.world_position, radius=20, impulse_per_length=-30)

    def aabb(self):
        return b2d.aabb(
            lower_bound=(-self.outer_box_radius, -self.outer_box_radius),
            upper_bound=(self.outer_box_radius, self.outer_box_radius),
        )


if __name__ == "__main__":
    Ragdoll.run(frontend_settings=dict(simple_ui=True, autostart=False))

## WorldOfGoo-like game

This example is inspired by the classic game [World of Goo](https://en.wikipedia.org/wiki/World_of_Goo).
To keep things simple, we only implement a small subset of the original game mechanics.

The source code of this example can be found in `goo_game.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/) deployment.

In [ ]:
import pyb2d3 as b2d
import numpy as np
from pyb2d3_sandbox import SampleBase
import pyb2d3_sandbox.widgets as widgets

from dataclasses import dataclass

# some constants to not overcomplicate the example
BALL_RADIUS = 0.1
HOLE_RADIUS = 0.2
FORCE_VECTOR_DRAW_WIDTH = 0.05
MAX_FORCE_VECTOR_LENGTH = 2


class UserDataStore(object):
    def __init__(self):
        self.id = 0
        self.data = {}

    def add(self, data):
        self.id += 1
        self.data[self.id] = data
        return self.id

    def __getitem__(self, key):
        if key == 0:
            return None
        return self.data.get(key, None)

    def __delitem__(self, key):
        if key == 0:
            raise KeyError("Cannot delete key 0 from UserData")
        if key in self.data:
            del self.data[key]
        else:
            raise KeyError(f"Key {key} not found in UserData")


class Goo(object):
    def __init__(self, sample):
        self.sample = sample
        self.world = sample.world
        self.ud = sample.ud
        self.body = None
        self.hertz = 4.0
        self.gravity_scale = 1.0
        self.density = 1.0
        self.friction = 0.5
        self.restitution = 0.2
        self.radius = 0.5
        self.enable_spring = True
        self.proto_edge_length = 2  # prototypical edge length for the goo ball
        self.discover_radius = self.proto_edge_length
        self.as_edge_max_distance = self.proto_edge_length
        self.as_goo_max_distance = (
            self.proto_edge_length * 2
        ) ** 2  # maximum distance to place as goo ball
        self.auto_expand = False
        self.connections = []  # list of connections to other goo balls
        self.max_degree = 40  # maximum number of connections per goo ball
        self.place_as_edge_squared_distance_threshold = (
            self.radius / 2
        ) ** 2  # threshold for placing as edge

    @property
    def debug_draw(self):
        return self.sample.debug_draw

    def has_capacity(self):
        # check if this goo ball can add more connections
        return len(self.connections) < self.max_degree

    def degrees(self):
        # return the number of connections this goo ball has
        return len(self.connections)

    def is_object_between_me_and_point(self, world_point):
        assert self.body is not None, "Goo ball not created yet. Cannot check for objects."

        own_pos = self.body.position
        translation = (world_point[0] - own_pos[0], world_point[1] - own_pos[1])

        # cast a ray from the goo ball to the world point
        ray_result = self.world.cast_ray_closest(origin=own_pos, translation=translation)
        return ray_result.hit

    def can_be_placed_here(self, world, pos):
        assert self.body is None, "Goo ball already created. Cannot place again."
        goos = self.sample.find_all_goos_in_radius(pos, self.discover_radius * 2)
        if not goos:
            return (False, False, None, None)

        # filter out goos that cannot add more connections
        goos = [(goo, dist) for goo, dist in goos if goo.degrees() < goo.max_degree]

        # filter out goos where there is an object between the goo and the position
        goos = [(goo, dist) for goo, dist in goos if not goo.is_object_between_me_and_point(pos)]

        # check if we can place as edge
        if len(goos) >= 2:
            # compute the best mid point between all pairs of goo balls
            # and the "pos" point
            best_pair = None
            best_distance = float("inf")
            for i in range(len(goos)):
                goo_a = goos[i][0]

                for j in range(i + 1, len(goos)):
                    goo_b = goos[j][0]

                    if goo_a.is_conncted_to(goo_b):
                        continue

                    d = b2d.mid_point_squared_distance(
                        goo_a.get_position(), goo_b.get_position(), pos
                    )

                    if d < best_distance:
                        best_distance = d
                        best_pair = (goo_a, goo_b)

            if best_distance <= self.place_as_edge_squared_distance_threshold:
                # we can place as edge
                return (True, True, best_pair, pos)

        # check if we can place as ball
        if len(goos) >= 2:
            best_pair = None
            best_distance = float("inf")
            for i in range(len(goos)):
                goo_a = goos[i][0]

                for j in range(i + 1, len(goos)):
                    goo_b = goos[j][0]

                    if not goo_a.is_conncted_to(goo_b):
                        continue

                    d = goos[i][1] + goos[j][1]
                    if d < best_distance:
                        best_distance = d
                        best_pair = (goo_a, goo_b)

            if best_distance <= self.as_goo_max_distance:
                # we can place as goo ball
                # return the two goo balls and the position
                return (True, False, best_pair, pos)

        return (False, False, None, None)

    def get_position(self):
        if self.body is not None:
            return self.body.position
        return None

    def create(self, world, pos):
        # create a dynamic body with a circle shape
        self.body = world.create_dynamic_body(
            position=pos, gravity_scale=self.gravity_scale, fixed_rotation=True
        )

        material = b2d.surface_material(
            restitution=self.restitution,
            friction=self.friction,
            custom_color=type(self).color_hex,
        )
        self.body.create_shape(
            b2d.shape_def(density=self.density, material=material, enable_contact_events=False),
            b2d.circle(radius=self.radius),
        )
        # set the user data to the Goo object itself
        self.body.user_data = self.ud.add(self)

        return self

    def connect(self, other_goo):
        # connect this goo ball to another goo ball with a distance joint
        if self.body is None or other_goo.body is None:
            raise ValueError("Both goo balls must be created before connecting them.")

        body_a = self.body
        body_b = other_goo.body

        if not self.auto_expand:
            edge_length = body_a.get_distance_to(body_b.position)
        else:
            edge_length = self.proto_edge_length

        joint_def = b2d.distance_joint_def(
            body_a=self.body,
            body_b=other_goo.body,
            length=edge_length,
            hertz=self.hertz,
            damping_ratio=0.5,
            enable_spring=self.enable_spring,
            collide_connected=True,
        )
        j = self.body.world.create_joint(joint_def)
        # set the user data to the joint

        # we store a "direction" st. we only
        # draw the edges once
        self.connections.append((other_goo, j, True))
        other_goo.connections.append((self, j, False))

    def is_conncted_to(self, other_goo):
        # check if this goo ball is connected to another goo ball
        for item in self.connections:
            if item[0] == other_goo:
                return True
        return False

    def draw_at(self, world_pos, angle=0):
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(world_pos),
            radius=self.radius,
            color=type(self).color_rgb,
        )
        self.debug_draw.draw_circle(world_pos, radius=self.radius, color=(0, 0, 0))

        eye_radius = self.radius / 4
        eye_offset = self.radius / 2.5
        local_left_eye_pos = (-eye_offset, -eye_offset / 2)
        local_right_eye_pos = (eye_offset, -eye_offset / 2)
        local_left_pupil_pos = (
            local_left_eye_pos[0] + eye_radius / 2,
            local_left_eye_pos[1],
        )
        local_right_pupil_pos = (
            local_right_eye_pos[0] + eye_radius / 2,
            local_right_eye_pos[1],
        )

        if self.body is not None:
            # get eye position in world coordinates
            left_eye_pos = self.body.world_point(local_left_eye_pos)
            right_eye_pos = self.body.world_point(local_right_eye_pos)
            left_pupil_pos = self.body.world_point(local_left_pupil_pos)
            right_pupil_pos = self.body.world_point(local_right_pupil_pos)
        else:
            left_eye_pos = world_pos + local_left_eye_pos
            right_eye_pos = world_pos + local_right_eye_pos
            left_pupil_pos = world_pos + local_left_pupil_pos
            right_pupil_pos = world_pos + local_right_pupil_pos

        # draw the eyes
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(left_eye_pos),
            radius=eye_radius,
            color=(255, 255, 255),
        )
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(right_eye_pos),
            radius=eye_radius,
            color=(255, 255, 255),
        )
        # draw the pupils
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(left_pupil_pos),
            radius=eye_radius / 2,
            color=(0, 0, 0),
        )
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(right_pupil_pos),
            radius=eye_radius / 2,
            color=(0, 0, 0),
        )

    def draw(self):
        assert self.body is not None, "Goo ball not created yet. Cannot draw."
        self.draw_at(self.body.position, self.body.angle)

    def draw_edge(self, p1, p2):
        self.debug_draw.draw_segment(p1, p2, color=(100, 100, 100))

    def draw_tentative_as_goo(self, pos, other_goos):
        for goo in other_goos:
            # draw the edge to the other goo ball
            self.draw_edge(goo.get_position(), pos)

        # draw the goo via draw_at
        self.draw_at(pos)

    def draw_tentative_as_edge(self, goo_a, goo_b):
        # draw via draw_edge
        self.draw_edge(goo_a.get_position(), goo_b.get_position())


class BlueGoo(Goo):
    color_rgb = (50, 50, 255)  # dark blue
    color_hex = b2d.hex_color(*color_rgb)
    text_color = (255, 255, 255)  # white text color
    name = "Plain Goo"

    def __init__(self, sample):
        super().__init__(sample)

    def draw_edge(self, p1, p2):
        self.debug_draw.draw_segment(p1=p1, p2=p2, color=BlueGoo.color_rgb)


class WhiteGoo(Goo):
    color_rgb = (200, 200, 200)  # light gray
    color_hex = b2d.hex_color(*color_rgb)
    text_color = (0, 0, 0)  # black text color
    name = "Drip Goo"

    def __init__(self, sample):
        super().__init__(sample)
        self.max_degree = 2  # white goo can connect to two other goo ball, ie forming line segments
        self.hertz = 2.0  # white goo is very elastic
        self.density = 0.25  # white goo is very light

    def draw_edge(self, p1, p2):
        self.debug_draw.draw_segment(p1=p1, p2=p2, color=WhiteGoo.color_rgb)

    def can_be_placed_here(self, world, pos):
        assert self.body is None, "Goo ball already created. Cannot place again."
        goos = self.sample.find_all_goos_in_radius(pos, self.discover_radius * 2)

        close_goo = None
        closest_distance = float("inf")

        for goo, dist in goos:
            # check if ther is an object between the goo and the position
            if goo.is_object_between_me_and_point(pos):
                continue
            if goo.has_capacity():
                if dist < closest_distance:
                    closest_distance = dist
                    close_goo = goo
        if close_goo is not None:
            # we can place as goo ball
            return (True, False, (close_goo,), pos)
        return (False, False, None, None)


# ballon, can only be connected to a single other goo
class RedGoo(Goo):
    color_rgb = (255, 0, 0)  # red
    color_hex = b2d.hex_color(*color_rgb)
    text_color = (255, 255, 255)  # white text color
    name = "Ballon Goo"

    def __init__(self, sample):
        super().__init__(sample)
        self.density = 0.5  # less dense than blue goo
        self.gravity_scale = -1.0
        self.hertz = 8.0
        self.enable_spring = False
        self.max_degree = 1  # red goo can only connect to one other goo ball
        self.auto_expand = True

    def draw_edge(self, p1, p2):
        self.debug_draw.draw_segment(
            p1=p1,
            p2=p2,
            # rope like color (ie brown orangish)
            color=(255, 100, 0),
        )

    def can_be_placed_here(self, world, pos):
        assert self.body is None, "Goo ball already created. Cannot place again."
        goos = self.sample.find_all_goos_in_radius(pos, self.discover_radius * 2)

        close_goo = None
        closest_distance = float("inf")

        for goo, dist in goos:
            if goo.is_object_between_me_and_point(pos):
                continue
            if goo.has_capacity():
                if dist < closest_distance:
                    closest_distance = dist
                    close_goo = goo
        if close_goo is not None:
            # we can place as goo ball
            return (True, False, (close_goo,), pos)
        return (False, False, None, None)


class BlackGoo(Goo):
    color_rgb = (10, 10, 0)  # almost black
    color_hex = b2d.hex_color(*color_rgb)
    text_color = (255, 255, 255)  # white text color
    name = "Heavy Goo"

    def __init__(self, sample):
        super().__init__(sample)
        self.density = 15
        self.hertz = 10  # very stiff

    def draw_edge(self, p1, p2):
        self.debug_draw.draw_segment(
            p1=p1,
            p2=p2,
            color=BlackGoo.color_rgb,
        )


class GooGame(SampleBase):
    @dataclass
    class Settings(SampleBase.Settings):
        current_level: int = 0

    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        # goo classes
        self.goo_classes = [BlueGoo, RedGoo, WhiteGoo, BlackGoo]
        self.goo_name_to_index = {goo_cls.name: i for i, goo_cls in enumerate(self.goo_classes)}

        self.selected_goo_cls = BlueGoo

        # the user data store
        self.ud = UserDataStore()

        # some state
        self.next_goo = self.selected_goo_cls(self)
        self.tentative_placement = (False, None, None, None)
        self.mouse_is_down = False
        self.drag_camera = False

        # add ui-elements
        self.frontend.add_widget(
            widgets.RadioButtons(
                label="Goo Type",
                options=[goo_cls.name for goo_cls in self.goo_classes],
                value=self.selected_goo_cls.name,
                callback=lambda goo_name: self.on_goo_change(self.goo_name_to_index[goo_name]),
            )
        )

    def on_goo_change(self, new_goo_type):
        # change the next goo type
        self.selected_goo_cls = self.goo_classes[new_goo_type]
        self.next_goo = self.selected_goo_cls(self)

    def on_mouse_down(self, event):
        self.mouse_is_down = True
        # self.last_canvas_pos = event.canvas_position
        self.tentative_placement = self.next_goo.can_be_placed_here(
            self.world, event.world_position
        )

        # if we cannot place the goo ball, we can drag the camera
        if not self.tentative_placement[0]:
            self.drag_camera = True

    def on_mouse_move(self, event):
        if self.mouse_is_down:
            world_point = event.world_position
            if self.drag_camera:
                # drag the camera
                self.frontend.drag_camera(event.world_delta)
            else:
                self.tentative_placement = self.next_goo.can_be_placed_here(self.world, world_point)
        # self.last_canvas_pos = p

    def on_mouse_up(self, event):
        self.mouse_is_down = False
        self.drag_camera = False
        world_point = event.world_position
        self.tentative_placement = self.next_goo.can_be_placed_here(self.world, world_point)
        if self.tentative_placement[0]:
            as_edge, goo_pair = self.tentative_placement[1], self.tentative_placement[2]
            if as_edge:
                # place as edge
                self.place_as_goo_edge(goo_pair[0], goo_pair[1])
            else:
                # place as ball
                self.place_as_goo(goo_pair, world_point)

            self.tentative_placement = (False, None, None)

    def aabb(self):
        margin = 5
        min_x = float("inf")
        max_x = float("-inf")
        min_y = float("inf")
        max_y = float("-inf")
        for goo in self.goo_balls:
            if goo.body is not None:
                pos = goo.get_position()
                margin = max(margin, goo.radius + 0.1)
                min_x = min(min_x, pos[0] - goo.radius - margin)
                max_x = max(max_x, pos[0] + goo.radius + margin)
                min_y = min(min_y, pos[1] - goo.radius - margin)
                max_y = max(max_y, pos[1] + goo.radius + margin)

        # aabb with margin
        return b2d.aabb(
            lower_bound=(min_x - margin, min_y - margin),
            upper_bound=(max_x + margin, max_y + margin),
        )

    # place as goo ball method
    def place_as_goo(self, goos_to_connect, world_point):
        self.next_goo.create(self.world, world_point)
        for g in goos_to_connect:
            self.next_goo.connect(g)
        self.goo_balls.append(self.next_goo)
        self.next_goo = self.selected_goo_cls(self)

    def place_as_goo_edge(self, goo_a, goo_b):
        goo_a.connect(goo_b)
        self.next_goo = self.selected_goo_cls(self)

    def find_all_goos_in_radius(self, pos, radius):
        ud = self.ud
        aabb = b2d.aabb_arround_point(point=pos, radius=radius)
        result = []

        square_radius = radius * radius

        def callback(shape):
            goo = ud[shape.body.user_data]
            if isinstance(goo, Goo):
                goo_pos = goo.get_position()
                distance_squared = (goo_pos[0] - pos[0]) ** 2 + (goo_pos[1] - pos[1]) ** 2
                if distance_squared <= square_radius:
                    result.append((goo, distance_squared))
            return True  # <-- continue searching

        self.world.overlap_aabb(aabb, callback)
        return result

    def post_debug_draw(self):
        # draw edges
        for goo in self.goo_balls:
            for other_goo, joint, created_edge in goo.connections:
                if created_edge:
                    goo.draw_edge(goo.get_position(), other_goo.get_position())

        if self.tentative_placement[0]:
            placable, as_edge, other_goos, pos = self.tentative_placement
            if as_edge:
                # draw the edge between the two goo balls
                goo_a, goo_b = other_goos
                self.next_goo.draw_tentative_as_edge(goo_a, goo_b)
            else:
                self.next_goo.draw_tentative_as_goo(pos, other_goos)

        # draw goos
        for goo in self.goo_balls:
            goo.draw()


class Level1(GooGame):
    @dataclass
    class Settings(GooGame.Settings):
        pass

    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        self.outer_box_radius = 100

        vertices = np.array(
            [
                (-100, 100),
                (-100, 0),
                (5, 0),
                (5, -100),
                (20, -100),
                (20, 0),
                (60, 0),
                (60, 100),
            ]
        )[::-1]  # reverse the order to make it clockwise
        self.outer_vertices = vertices

        # attach the chain shape to a static body
        self.box_body = self.world.create_static_body(position=(0, 0))
        self.box_body.create_chain(b2d.chain_def(points=vertices, is_loop=True))

        goo_cls = BlueGoo

        g = goo_cls(self)
        edge_length = goo_cls(self).proto_edge_length
        r = g.radius
        # 3 goo balls as equilateral triangle with edge_length as the length of the edges
        self.goo_balls = [
            goo_cls(self).create(self.world, (-edge_length / 2, 0 + r)),
            goo_cls(self).create(self.world, (edge_length / 2, 0 + r)),
            goo_cls(self).create(self.world, (0, (edge_length / 2) * np.sqrt(3) + r)),
        ]
        # connect the goo balls
        self.goo_balls[0].connect(self.goo_balls[1])
        self.goo_balls[1].connect(self.goo_balls[2])
        self.goo_balls[2].connect(self.goo_balls[0])

        # if we are in a headless mode, add one more goo to make it a bit whobbly
        if self.frontend.settings.headless:
            g = goo_cls(self)
            g.create(self.world, (3.5, 3))
            self.goo_balls.append(g)

            self.goo_balls[1].connect(g)
            self.goo_balls[2].connect(g)


if __name__ == "__main__":
    Level1.run(frontend_settings=dict(simple_ui=True, autostart=False))

## Minigolf with a catch

This example is inspired by classic minigolf games, but instead of a single ball, we have to balls which are connected by a spring.
The source code of this example can be found in `coupled_minigolf.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/) deployment.

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase


import numpy as np
import time
from enum import Enum

from dataclasses import dataclass

# some constants to not overcomplicate the example
BALL_RADIUS = 0.1
HOLE_RADIUS = 0.2
MAX_FORCE_VECTOR_LENGTH = 2
BALL_COLORS = [b2d.hex_color(100, 100, 220), b2d.hex_color(100, 100, 220)]


@dataclass
class CourseLevelData:
    # where are the 2 balls placed?
    start_positions: list[tuple]

    # where is the hole?
    hole_position: tuple

    # the polygons of the course
    course_polygons: list[list[tuple]]


eps = 0.25
Levels = [
    CourseLevelData(
        start_positions=[(1, 1), (1, 1.8)],
        hole_position=(5, 1),
        course_polygons=[[(0, 0), (0, 2), (6, 2), (6, 0)]],
    ),
    CourseLevelData(
        start_positions=[(1, 1), (1, 1.8)],
        hole_position=(5, -1),
        course_polygons=[
            [
                (0, 0),
                (0, 2),
                (6, 2),
                (6, -2),
                (4, -2),
                (4, 0),
            ]
        ],
    ),
    CourseLevelData(
        start_positions=[(1, 1.75), (1, 2.8)],
        hole_position=(1, 5),
        course_polygons=[
            [(0, 0), (0, 2), (2.5, 2), (2.5, 4), (0, 4), (0, 6), (4, 6), (4, 0)],
            [
                (0 + eps, 2 + eps),
                (0 + eps, 4 - eps),
                (2 - eps, 4 - eps),
                (2 - eps, 2 + eps),
            ],
        ],
    ),
]


class GolfState(Enum):
    WAITING_FOR_BALL_CLICK = 0  # we are waiting for the user to click on a ball
    SET_DRAG_FORCE = 1  # user is setting the force via mouse drag/move
    WAITING_FOR_REST = 2  # we are waiting for the ball to come to rest
    BALL_IS_IN_HOLE = 3  # the ball is in the hole, we can proceed to the next level


class CoupledMinigolf(SampleBase):
    @dataclass
    class Settings(SampleBase.Settings):
        current_level: int = 0

    def __init__(self, frontend, settings):
        super().__init__(frontend, settings.set_gravity((0, 0)))
        self.outer_box_radius = 100

        # data for the current level
        self.level = Levels[settings.current_level]

        # create the chain shape for the course
        chain_material = b2d.surface_material(restitution=0.75)
        for polygon in self.level.course_polygons:
            self.anchor_body.create_chain(
                b2d.chain_def(polygon, is_loop=True, material=chain_material)
            )

        # create the balls
        self.balls = []

        for i, position in enumerate(self.level.start_positions):
            body = self.world.create_dynamic_body(
                position=position,
                linear_damping=0.75,
                fixed_rotation=True,
            )
            material = b2d.surface_material(restitution=0.75, custom_color=BALL_COLORS[i])
            ball_shape_def = b2d.shape_def(density=1, material=material)
            body.create_shape(ball_shape_def, b2d.circle(radius=BALL_RADIUS))
            self.balls.append(body)
        self.balls_shape = [ball.shapes[0] for ball in self.balls]

        # create a distance joint between the two balls
        self.world.create_distance_joint(
            body_a=self.balls[0],
            body_b=self.balls[1],
            length=np.linalg.norm(
                np.array(self.balls[0].position) - np.array(self.balls[1].position)
            ),
            collide_connected=True,
            hertz=0.35,
            enable_spring=True,
            damping_ratio=0.25,
        )

        # state of the game
        self.state = GolfState.WAITING_FOR_BALL_CLICK
        self.dragged_ball_index = None
        self.drag_pos = None
        self.force_vector_length = 0
        self.ball_in_hole_time = None

        if self.frontend.settings.headless:
            self.balls[0].apply_linear_impulse_to_center((0.2, 0.01), wake=True)
            self.state = GolfState.WAITING_FOR_REST

    def on_mouse_down(self, event):
        if self.state == GolfState.WAITING_FOR_BALL_CLICK:
            # check if the user clicked on one of the balls
            for i, ball in enumerate(self.balls):
                if self.balls_shape[i].test_point(event.world_position):
                    self.state = GolfState.SET_DRAG_FORCE
                    self.dragged_ball_index = i
                    self.drag_pos = event.world_position
                    return

    def on_mouse_up(self, event):
        if self.state == GolfState.SET_DRAG_FORCE:
            # we are setting the force, so we apply it now
            dragged_ball = self.balls[self.dragged_ball_index]
            force_vector = np.array(self.drag_pos) - np.array(dragged_ball.position)
            force_vector *= -15
            dragged_ball.apply_force_to_center(force_vector, True)
            self.state = GolfState.WAITING_FOR_REST

    def on_mouse_move(self, event):
        if self.state == GolfState.SET_DRAG_FORCE:
            raw_drag_pos = event.world_position

            # limit the length of the force vector
            force_vector = np.array(raw_drag_pos) - np.array(
                self.balls[self.dragged_ball_index].position
            )
            force_vector_length = np.linalg.norm(force_vector)
            self.force_vector_length = force_vector_length
            if force_vector_length > MAX_FORCE_VECTOR_LENGTH:
                self.force_vector_length = MAX_FORCE_VECTOR_LENGTH
                force_vector = force_vector / force_vector_length * MAX_FORCE_VECTOR_LENGTH
                self.drag_pos = (
                    np.array(self.balls[self.dragged_ball_index].position) + force_vector
                )
            else:
                self.drag_pos = raw_drag_pos

    def balls_rest(self):
        mag = self.balls[0].linear_velocity_magnitude() + self.balls[1].linear_velocity_magnitude()
        return mag < 0.0001

    def post_update(self, dt):
        # if the ball is already in the hole, check if all balls are at rest and
        #
        if self.state == GolfState.BALL_IS_IN_HOLE:
            if self.balls_rest():
                # proceed to the next level (jump to first level if we are at the last one)
                self.settings.current_level += 1
                if self.settings.current_level >= len(Levels):
                    self.settings.current_level = 0

                self.frontend.set_sample(CoupledMinigolf, self.settings)

            return

        # check if the center of the hole is inside the balls
        for i, ball in enumerate(self.balls):
            if self.balls_shape[i].test_point(self.level.hole_position):
                self.state = GolfState.BALL_IS_IN_HOLE
                self.ball_in_hole_time = time.time()

                hole_body = self.world.create_static_body(position=self.level.hole_position)

                # create a distance joint to capture the ball in the hole
                self.world.create_distance_joint(
                    body_a=ball,
                    body_b=hole_body,
                    length=0.01,  # very small length to keep the ball in place
                    enable_spring=True,
                    hertz=100,
                    damping_ratio=1.0,
                )
                return

        if self.state == GolfState.WAITING_FOR_REST:
            # check if balls are at rest
            if self.balls_rest():
                self.state = GolfState.WAITING_FOR_BALL_CLICK

    # to ensure the hole is **under** the balls
    # we use the pre_debug_draw method
    def pre_debug_draw(self):
        self.debug_draw.draw_solid_circle(
            transform=b2d.transform(self.level.hole_position),
            radius=HOLE_RADIUS,
            color=b2d.hex_color(0, 0, 0),
        )

    # draw above the debgug draw
    def post_debug_draw(self):
        if self.state == GolfState.SET_DRAG_FORCE:
            # liner interolate force vector color from yellow to red
            color_yellow = np.array([255, 255, 0])
            color_red = np.array([255, 0, 0])
            color = color_yellow + (color_red - color_yellow) * (
                self.force_vector_length / MAX_FORCE_VECTOR_LENGTH
            )
            # round colors to integers
            color = (round(color[0]), round(color[1]), round(color[2]))

            # draw a line from the ball to the mouse position
            dragged_ball = self.balls[self.dragged_ball_index]
            self.debug_draw.draw_segment(
                dragged_ball.position,
                self.drag_pos,
                color=b2d.hex_color(*color),
            )
            for pos in [dragged_ball.position, self.drag_pos]:
                self.debug_draw.draw_solid_circle(
                    transform=b2d.transform(pos),
                    color=b2d.hex_color(*color),
                    radius=BALL_RADIUS * 0.5,
                )

    def aabb(self):
        # bouding box from self.levels.course_polygons
        min_x = min(p[0] for polygon in self.level.course_polygons for p in polygon)
        min_y = min(p[1] for polygon in self.level.course_polygons for p in polygon)
        max_x = max(p[0] for polygon in self.level.course_polygons for p in polygon)
        max_y = max(p[1] for polygon in self.level.course_polygons for p in polygon)
        margin = 1
        return b2d.aabb(
            lower_bound=(min_x - margin, min_y - margin),
            upper_bound=(max_x + margin, max_y + margin),
        )


if __name__ == "__main__":
    CoupledMinigolf.run(frontend_settings=dict(simple_ui=True, autostart=False))

## More Examples

Many more examples can be found at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/) deployment where you can also edit and run the examples directly in your browser and create your own games and simultions.